In [1]:
# Libraries                                                                                # Version

import os

import pandas as pd                                                                            # 1.3.4
import numpy as np                                                                             # 1.20.3
import scipy.stats as stats                                                                    # 0.12.2

from sklearn.linear_model import LogisticRegression                                            # 0.24.2
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier, plot_tree
# from IPython.display import Image 
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score 
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score 
from sklearn.metrics import roc_curve, auc

# from sklearn.model_selection import cross_val_predict
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler #, OneHotEncoder
import matplotlib.pyplot as plt 

# Load & Clean the raw data

Weicheng Cao

In [2]:
# Load raw data
os.chdir('/Users/caoweicheng/Desktop/UF书籍/AI and Machine Learning/Assignment 2 NW')
raw = pd.read_excel('LendingClub_Extended_DataSet-4.xlsx')

# check raw data
print("Size of the dataset (row, col): ", raw.shape)
raw.head()

Size of the dataset (row, col):  (25000, 39)


,Unnamed: 0,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,...,bc_util,chargeoff_within_12_mths,mort_acc,mths_since_recent_bc,mths_since_recent_inq,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,0,20000.0,60 months,17.93,1 year,MORTGAGE,44304.0,Verified,Charged Off,debt_consolidation,...,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,1,30000.0,36 months,11.99,10+ years,MORTGAGE,136000.0,Verified,Current,debt_consolidation,...,42.6,0.0,3.0,40.0,3.0,1.0,116075.0,69594.0,27500.0,83175.0
2,2,21500.0,36 months,11.99,1 year,RENT,50000.0,Source Verified,Fully Paid,debt_consolidation,...,66.8,0.0,2.0,23.0,6.0,0.0,116920.0,74939.0,19700.0,78020.0
3,3,10000.0,36 months,13.67,NaN,RENT,64400.0,Source Verified,Fully Paid,debt_consolidation,...,50.9,0.0,1.0,3.0,NaN,0.0,39547.0,16824.0,11200.0,25547.0
4,4,5000.0,36 months,8.49,10+ years,MORTGAGE,88000.0,Source Verified,Current,debt_consolidation,...,92.6,0.0,1.0,14.0,8.0,0.0,104310.0,7582.0,3300.0,13015.0


In [3]:
raw.info() # Overview raw data information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Unnamed: 0                  25000 non-null  int64         
 1   loan_amnt                   24999 non-null  float64       
 2   term                        24999 non-null  object        
 3   int_rate                    24999 non-null  float64       
 4   emp_length                  23498 non-null  object        
 5   home_ownership              24999 non-null  object        
 6   annual_inc                  24999 non-null  float64       
 7   verification_status         24999 non-null  object        
 8   loan_status                 24999 non-null  object        
 9   purpose                     24999 non-null  object        
 10  title                       24633 non-null  object        
 11  addr_state                  24999 non-null  object    

## Delete the useless columns and rows

In [4]:
# delete the labeled "unnamed" column

raw=raw.iloc[:,1:] # will keep all the columns, starting from column #1

raw.head() #show the data after deleted

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,title,...,bc_util,chargeoff_within_12_mths,mort_acc,mths_since_recent_bc,mths_since_recent_inq,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,20000.0,60 months,17.93,1 year,MORTGAGE,44304.0,Verified,Charged Off,debt_consolidation,Todd's Loan,...,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,30000.0,36 months,11.99,10+ years,MORTGAGE,136000.0,Verified,Current,debt_consolidation,Debt consolidation,...,42.6,0.0,3.0,40.0,3.0,1.0,116075.0,69594.0,27500.0,83175.0
2,21500.0,36 months,11.99,1 year,RENT,50000.0,Source Verified,Fully Paid,debt_consolidation,Debt Be Gone,...,66.8,0.0,2.0,23.0,6.0,0.0,116920.0,74939.0,19700.0,78020.0
3,10000.0,36 months,13.67,NaN,RENT,64400.0,Source Verified,Fully Paid,debt_consolidation,Debt consolidation,...,50.9,0.0,1.0,3.0,NaN,0.0,39547.0,16824.0,11200.0,25547.0
4,5000.0,36 months,8.49,10+ years,MORTGAGE,88000.0,Source Verified,Current,debt_consolidation,Debt consolidation,...,92.6,0.0,1.0,14.0,8.0,0.0,104310.0,7582.0,3300.0,13015.0


In [5]:
# delete the NaN rows
raw = raw.dropna(axis=0, thresh=38) # delete the NaN rows that exceess 38 NaN data

# Size of data following this cleanup
print(raw.shape)

(9552, 38)


In [6]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9552 entries, 1 to 24999
Data columns (total 38 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   loan_amnt                   9552 non-null   float64       
 1   term                        9552 non-null   object        
 2   int_rate                    9552 non-null   float64       
 3   emp_length                  9552 non-null   object        
 4   home_ownership              9552 non-null   object        
 5   annual_inc                  9552 non-null   float64       
 6   verification_status         9552 non-null   object        
 7   loan_status                 9552 non-null   object        
 8   purpose                     9552 non-null   object        
 9   title                       9552 non-null   object        
 10  addr_state                  9552 non-null   object        
 11  dti                         9552 non-null   float64    

In [7]:
# While we can use all the data in the dataframe for the  DecisionTreeClassifier, some entries will be 
# more problematic for the (logistic) regression. For example, 'addr_state' can take on any one the 50 states abbreviations.
# We will therefore drop 'addr_state' from the data.

# Similarly, 'earliest_cr_line', which is formatted as date may be handled with a bit of trickery but to keep the assignment simple,
# 'earliest_cr_line' will also be dropped here.

raw = raw.drop(['addr_state', 'earliest_cr_line'], axis=1)

# Other features may be simplified, even if they are of object type (character strings), as will be shown below

In [8]:
# The feature 'title' is a bit complicated to handle here and will also be omitted
# But before this is done, here is what it looks like

raw['title'].unique()

array(['Debt consolidation', 'Credit card refinancing', 'Vacation',
       'Other', 'Major purchase', 'Home improvement', 'Medical expenses',
       'Home buying', 'Business', 'Car financing',
       'Moving and relocation', 'Green loan'], dtype=object)

In [9]:
# drop the "title" col
raw = raw.drop(['title'], axis=1)

In [10]:
# At this stage, this is what the data look like
raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9552 entries, 1 to 24999
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   9552 non-null   float64
 1   term                        9552 non-null   object 
 2   int_rate                    9552 non-null   float64
 3   emp_length                  9552 non-null   object 
 4   home_ownership              9552 non-null   object 
 5   annual_inc                  9552 non-null   float64
 6   verification_status         9552 non-null   object 
 7   loan_status                 9552 non-null   object 
 8   purpose                     9552 non-null   object 
 9   dti                         9552 non-null   float64
 10  delinq_2yrs                 9552 non-null   float64
 11  fico_range_low              9552 non-null   float64
 12  fico_range_high             9552 non-null   float64
 13  inq_last_6mths              9552

In [11]:
# As an example, in the first row:
raw.iloc[1]

loan_amnt                                 5000.0
term                                   36 months
int_rate                                    8.49
emp_length                             10+ years
home_ownership                          MORTGAGE
annual_inc                               88000.0
verification_status              Source Verified
loan_status                              Current
purpose                       debt_consolidation
dti                                         5.32
delinq_2yrs                                  0.0
fico_range_low                             660.0
fico_range_high                            664.0
inq_last_6mths                               0.0
open_acc                                     4.0
revol_bal                                 3055.0
revol_util                                  92.6
total_acc                                    5.0
acc_now_delinq                               0.0
tot_cur_bal                              77053.0
open_acc_6m         

In [12]:
# Let's look at the possible values of 'verification status'
raw['verification_status'].unique()

array(['Verified', 'Source Verified', 'Not Verified'], dtype=object)

## Numeric status by 0 and 1

In [13]:
# As is, 'verification_status' may not be a problem for the DecisionTreeClassifier but is for the regression approach
# Let's therefore convert it to a (numeric) categorical variable
# Here we will contrast between "Verified" and "Source Verified", on one hand, with "Not Verified", on the other
raw['verification_status'] = raw['verification_status'].replace(['Verified','Source Verified'],1)

In [14]:
# numeric status by only 0 and 1
raw['verification_status'] = raw['verification_status'].replace(['Not Verified'],0)

In [15]:
# check the numeric status
raw['verification_status']

1        1
4        1
5        1
6        0
8        1
        ..
24990    1
24991    1
24992    1
24993    1
24999    1
Name: verification_status, Length: 9552, dtype: int64

In [16]:
# At this stage, this is what the data consists of
raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9552 entries, 1 to 24999
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   9552 non-null   float64
 1   term                        9552 non-null   object 
 2   int_rate                    9552 non-null   float64
 3   emp_length                  9552 non-null   object 
 4   home_ownership              9552 non-null   object 
 5   annual_inc                  9552 non-null   float64
 6   verification_status         9552 non-null   int64  
 7   loan_status                 9552 non-null   object 
 8   purpose                     9552 non-null   object 
 9   dti                         9552 non-null   float64
 10  delinq_2yrs                 9552 non-null   float64
 11  fico_range_low              9552 non-null   float64
 12  fico_range_high             9552 non-null   float64
 13  inq_last_6mths              9552

In [17]:
# Let's check the values associated with 'purpose'
raw['purpose'].unique()

array(['debt_consolidation', 'credit_card', 'vacation', 'other',
       'major_purchase', 'home_improvement', 'medical', 'house',
       'small_business', 'car', 'moving', 'renewable_energy'],
      dtype=object)

In [18]:
raw['purpose']

1        debt_consolidation
4        debt_consolidation
5        debt_consolidation
6        debt_consolidation
8               credit_card
                ...        
24990    debt_consolidation
24991              vacation
24992        small_business
24993    debt_consolidation
24999        small_business
Name: purpose, Length: 9552, dtype: object

In [19]:
# Again, with the regression in mind, let's make a model that will contrast between loan consolidation and
# other purposes for the loan
raw.loc[raw['purpose']=='debt_consolidation', 'purpose']=1
raw.loc[(raw['purpose']!=1) &(raw['purpose']!='debt_consolidation'), 'purpose']=0

In [20]:
raw['purpose']

1        1
4        1
5        1
6        1
8        0
        ..
24990    1
24991    0
24992    0
24993    1
24999    0
Name: purpose, Length: 9552, dtype: object

In [21]:
# 'term' is non-numeric object. Let's see how many unique terms there are in the data
raw['term'].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [22]:
# Since there are only two term options (36 months and 60 months), we can use them as is for the DecisionTreeClassifier approach but we 
# we will need to make them categorical (a.k.a. dummy) variables for (logistic) regression

raw = pd.get_dummies(raw, columns=['term'],  drop_first=True) # dummies variables

raw.head()

,loan_amnt,int_rate,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,...,chargeoff_within_12_mths,mort_acc,mths_since_recent_bc,mths_since_recent_inq,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,term_ 60 months
1,30000.0,11.99,10+ years,MORTGAGE,136000.0,1,Current,1,20.63,3.0,...,0.0,3.0,40.0,3.0,1.0,116075.0,69594.0,27500.0,83175.0,0
4,5000.0,8.49,10+ years,MORTGAGE,88000.0,1,Current,1,5.32,0.0,...,0.0,1.0,14.0,8.0,0.0,104310.0,7582.0,3300.0,13015.0,0
5,16150.0,30.74,10+ years,RENT,38500.0,1,Charged Off,1,33.73,0.0,...,0.0,0.0,9.0,1.0,0.0,64635.0,46857.0,11400.0,41935.0,1
6,18400.0,14.08,8 years,RENT,40000.0,0,Current,1,19.11,0.0,...,0.0,0.0,5.0,9.0,0.0,49719.0,21706.0,42400.0,0.0,0
8,35000.0,12.79,9 years,MORTGAGE,118000.0,1,Fully Paid,0,26.66,0.0,...,0.0,6.0,9.0,4.0,0.0,625456.0,116751.0,97800.0,22200.0,0


In [23]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9552 entries, 1 to 24999
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   9552 non-null   float64
 1   int_rate                    9552 non-null   float64
 2   emp_length                  9552 non-null   object 
 3   home_ownership              9552 non-null   object 
 4   annual_inc                  9552 non-null   float64
 5   verification_status         9552 non-null   int64  
 6   loan_status                 9552 non-null   object 
 7   purpose                     9552 non-null   object 
 8   dti                         9552 non-null   float64
 9   delinq_2yrs                 9552 non-null   float64
 10  fico_range_low              9552 non-null   float64
 11  fico_range_high             9552 non-null   float64
 12  inq_last_6mths              9552 non-null   float64
 13  open_acc                    9552

In [24]:
raw.iloc[1]

loan_amnt                        5000.0
int_rate                           8.49
emp_length                    10+ years
home_ownership                 MORTGAGE
annual_inc                      88000.0
verification_status                   1
loan_status                     Current
purpose                               1
dti                                5.32
delinq_2yrs                         0.0
fico_range_low                    660.0
fico_range_high                   664.0
inq_last_6mths                      0.0
open_acc                            4.0
revol_bal                        3055.0
revol_util                         92.6
total_acc                           5.0
acc_now_delinq                      0.0
tot_cur_bal                     77053.0
open_acc_6m                         0.0
max_bal_bc                       3055.0
all_util                           47.0
total_rev_hi_lim                 3300.0
acc_open_past_24mths                1.0
bc_util                            92.6


In [25]:
# As is, 'loan_status' has more than one value. This will not be a problem for the DecisionTreeClassifier 
# but will be for regression. Let's look at the values:
raw['loan_status'].unique()

array(['Current', 'Charged Off', 'Fully Paid', 'Late (31-120 days)',
       'Late (16-30 days)', 'In Grace Period'], dtype=object)

In [26]:
# As we did with 'purpose', we can also categorize 'loan_status' in to a numeric form, contrasting, say, 'Current' and 'Fully Paid'
# vs. all the others
# Again, with the regression in mind, let's make a model that will contrast between loan consolidation and
# other puposes for the loan

raw.loc[(raw['loan_status']=='Current') | (raw['loan_status']=='Fully Paid'), 'loan_status']=1
raw.loc[(raw['loan_status']!=1) & (raw['loan_status']!='Current') &  (raw['loan_status']!='Fully Paid'), 'loan_status']=0

In [27]:
# As an example, let's check the first row 
raw.iloc[0]

loan_amnt                       30000.0
int_rate                          11.99
emp_length                    10+ years
home_ownership                 MORTGAGE
annual_inc                     136000.0
verification_status                   1
loan_status                           1
purpose                               1
dti                               20.63
delinq_2yrs                         3.0
fico_range_low                    670.0
fico_range_high                   674.0
inq_last_6mths                      1.0
open_acc                           12.0
revol_bal                       13383.0
revol_util                         40.7
total_acc                          34.0
acc_now_delinq                      0.0
tot_cur_bal                     69594.0
open_acc_6m                         1.0
max_bal_bc                      10598.0
all_util                           60.0
total_rev_hi_lim                32900.0
acc_open_past_24mths                6.0
bc_util                            42.6


In [28]:
# Let's now look at the values of 'home_ownership'
raw['home_ownership'].unique()

array(['MORTGAGE', 'RENT', 'OWN', 'ANY'], dtype=object)

In [29]:
# Here we, again, for the regression, we want to convert 'home_owership' to a numeric format
# We could choose a binary form, contrasting whether it does not matter (i.e., 'ANY') vs all the others, or
# we could distinguish between all answers, using the function 'get_dummies'
raw = pd.get_dummies(raw, columns=['home_ownership'],  drop_first=True)

In [30]:
# Let's check again the first row 
raw.iloc[0]

loan_amnt                       30000.0
int_rate                          11.99
emp_length                    10+ years
annual_inc                     136000.0
verification_status                   1
loan_status                           1
purpose                               1
dti                               20.63
delinq_2yrs                         3.0
fico_range_low                    670.0
fico_range_high                   674.0
inq_last_6mths                      1.0
open_acc                           12.0
revol_bal                       13383.0
revol_util                         40.7
total_acc                          34.0
acc_now_delinq                      0.0
tot_cur_bal                     69594.0
open_acc_6m                         1.0
max_bal_bc                      10598.0
all_util                           60.0
total_rev_hi_lim                32900.0
acc_open_past_24mths                6.0
bc_util                            42.6
chargeoff_within_12_mths            0.0


In [31]:
# Finally, though employment length is understood to be numerical, it appears in the data in object form (character string).
# Lets' first look at its possible values
raw['emp_length'].unique()

array(['10+ years', '8 years', '9 years', '< 1 year', '5 years',
       '2 years', '3 years', '7 years', '1 year', '6 years', '4 years'],
      dtype=object)

In [32]:
# The simplest way to handle this feature is to remove the word "years" or "year", and replace 10+ by 10

raw['emp_length'] = raw['emp_length'].str.replace('years','')
raw['emp_length'] = raw['emp_length'].str.replace('year','')
raw.loc[(raw['emp_length']=='10+ '), 'emp_length']=10

In [33]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9552 entries, 1 to 24999
Data columns (total 37 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   9552 non-null   float64
 1   int_rate                    9552 non-null   float64
 2   emp_length                  9552 non-null   object 
 3   annual_inc                  9552 non-null   float64
 4   verification_status         9552 non-null   int64  
 5   loan_status                 9552 non-null   object 
 6   purpose                     9552 non-null   object 
 7   dti                         9552 non-null   float64
 8   delinq_2yrs                 9552 non-null   float64
 9   fico_range_low              9552 non-null   float64
 10  fico_range_high             9552 non-null   float64
 11  inq_last_6mths              9552 non-null   float64
 12  open_acc                    9552 non-null   float64
 13  revol_bal                   9552

## Remove target col for features

In [34]:
# remove target column to create feature only dataset
X_data = raw.drop('loan_status', axis=1)

# store target column
Y_data = raw['loan_status']

print(X_data.shape, Y_data.shape)

(9552, 36) (9552,)


In [35]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9552 entries, 1 to 24999
Data columns (total 37 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   9552 non-null   float64
 1   int_rate                    9552 non-null   float64
 2   emp_length                  9552 non-null   object 
 3   annual_inc                  9552 non-null   float64
 4   verification_status         9552 non-null   int64  
 5   loan_status                 9552 non-null   object 
 6   purpose                     9552 non-null   object 
 7   dti                         9552 non-null   float64
 8   delinq_2yrs                 9552 non-null   float64
 9   fico_range_low              9552 non-null   float64
 10  fico_range_high             9552 non-null   float64
 11  inq_last_6mths              9552 non-null   float64
 12  open_acc                    9552 non-null   float64
 13  revol_bal                   9552

# Model 1: Logistic Regression

In [36]:
# plotting packages
get_ipython().run_line_magic('matplotlib', 'inline')
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as clrs

# Kmeans algorithm from scikit-learn
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

## Split data for model 1

## Clean data for model 1

In [37]:
#clean data for model 1
df1 =raw.iloc[:,:-4]
df1 = df1.drop(["purpose"], axis=1) # drop "purpose"


# Specify the target variable and the feature variables
y = df1['loan_status'].astype(int)
X = df1.drop('loan_status', axis=1)

In [38]:
# Split the data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2023) #Split function

## Create and Train a Logistic Regression Model

In [39]:
# Create a Logistic Regression Model
log_reg = LogisticRegression(max_iter=1000)

# Train the model 
log_reg.fit(X_train, y_train)

# make predictions on the test set
y_pred = log_reg.predicte(X_test)


ValueError: could not convert string to float: '< 1 '

## Set Threshold to divide the result

In [ ]:
# If the predicted probability is > 0.5, it belongs to class 1, else it belongs to class 0

y_pred = [1 if i > 0.5 else 0 for i in y_pred]

# Print the classification report
print(calssification_report(y_test, y_pred))

# Print the confusion matrix
print(confusion_matrix(y_test,y_pred))